In [ ]:
#$ export FLASK_APP=endpoints.py
#$ flask run
# http://127.0.0.1:5000/

In [17]:
import requests
from sql_caller import engine_connector, user_call, chat_checker
conn = engine_connector()
from sqlalchemy import text
import pandas as pd
from randomuser import RandomUser
from itertools import combinations
import random
import numpy as np
from datetime import datetime
import re

In [5]:
## Add users to DB

user_list = RandomUser.generate_users(40)

for user in user_list:
    users_dict = {'name':user.get_first_name(), 'last_name':user.get_last_name(), 'nick_name':user.get_username()}
    endpoint = "new_user?user_name={name}&user_surname={last_name}&user_nick={nick_name}".format(**users_dict)
    
    requests.get("http://127.0.0.1:5000/" + endpoint)

In [6]:
## Add chats

query = f"""SELECT user_nick FROM chat_api.users;
         """
users_list = pd.read_sql(con=conn, sql=query)['user_nick'].tolist()

selection = random.sample(users_list, 10)
chats_comb = list(combinations(selection, 2))
ten_chats = random.sample(chats_comb, 10)

for chat in ten_chats:
    chat_dict = {'sender_nick': chat[0], 'recv_nick': chat[1]}
    endpoint = "chat/create?sender_nick={sender_nick}&recv_nick={recv_nick}".format(**chat_dict)
    
    requests.get("http://127.0.0.1:5000/" + endpoint)

In [27]:
### Create a function to correct the strings
string = "Hello, it's me"
string = string.replace(" ", "_").replace("'", "")
string

endpoint = f"chat/addmessage?sender_nick=beautifulbird317&recv_nick=ticklishbutterfly899&message={string}"

requests.get("http://127.0.0.1:5000/" + endpoint)

<Response [200]>

In [28]:
string = "Hello, it's me"
string = string.replace(" ", "_").replace("'", "")
string

'Hello,_its_me'

In [29]:
message = 'Hello,_its_me'
message = string.replace("_", " ")
message

'Hello, its me'

In [ ]:
@app.route("/")
def hello_world():
    return {'hello':'world'}

@app.route('/name')
def te_doy_tu_nombre():
    name = request.args.get('name')
    lastname = request.args.get('lastname')
    return {"Hello": f"{name} {lastname}"}

@app.route('/mucho')
def mucho():
    args = request.args
    res = {}
    for k, v in args.items():
        res[k] = v
    return res

